In [113]:
# Install all the necessary python libraries if anaconda is not able to find them.
# !pip3 install google-maps-reviews
# Too bad the google-maps-reviews lib requires OutScraper API Key which is only free for the first 500 places and it requires minimum deposit of $500

# Expecting 5000++ of FNB listed

In [114]:
# Import all the necessary python libraries
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException


In [115]:
# Note: By default, The Safari WebDriver is included with the operating system. There is no additional installations required.
driver = webdriver.Safari()
driver.maximize_window()
url = 'https://www.google.com/maps'
driver.get(url)
time.sleep(5)

In [116]:
# Functions to return the element

# def find_web_element(locator_strategy="XPATH",locator_value=None,child_driver=None,iterate=False):

#     # Map string locator strategy to By class attributes
#     locator_strategy_map = {
#         'XPATH': By.XPATH,
#         'CSS': By.CSS_SELECTOR,
#         'ID': By.ID,
#         'NAME': By.NAME,
#         'CLASS': By.CLASS_NAME,
#         'TAG': By.TAG_NAME,
#         'LINK': By.LINK_TEXT,
#         'PARTIAL': By.PARTIAL_LINK_TEXT,
#     }
    
#     current_driver = child_driver or driver
#     # Default to By.XPATH if not found
#     by_locator = locator_strategy_map.get(locator_strategy.upper(), By.XPATH)
#     print(by_locator)

#     if iterate is False:
#         return current_driver.find_element(by_locator, locator_value)
#     else:
#         return current_driver.find_elements(by_locator, locator_value)

# def find_element_by_css(xpath):
#     return driver.find_element(By.XPATH, xpath)

# def find_element_by_class(class_name):
#     return driver.find_element(By.CLASS_NAME, class_name)

In [117]:
# Define CONSTANTS

FNB_CATEGORY_LIST = ['Halal Food', 'Western Food', 'Fast Food', 'Vegetarian Food', 
                     'Chinese Food', 'Indian Food', 'Malay Food', 'Middle Eastern Food', 
                     'Japanese Food', 'Korean Food', 'Cafe', 'Dessert'] #12

# Exclude -> Coffee, Bar, Bakery

AREA_LIST = ['Subang Jaya', 'Puchong', 'Seri Kembangan', 'Petaling Jaya', 'Cheras'] #5

# Exclude -> Klang, Old Klang Road, Kepong, Sri Petaling, Bukit Jalil

SEARCH_BOX_INPUT_XPATH = '//*[@id="searchboxinput"]'
CLOSE_BUTTON_XPATH = '//*[@id="searchbox"]/div[2]/button'
SCROLLABLE_COMPONENT_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]'
LOADING_SPINNER_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[146]/div/div'
END_OF_LIST_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[169]/div/p/span'
RESULT_LABEL_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/div[2]/div[2]/div[1]/h1'
RESULT_LIST_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]'

FNB_DETAILS_XPATH = '//a[@class="hfpxzc" and @jslog[starts-with(., "12690")]]'
FNB_OPENING_HOURS_BTN_DOWN_XPATH = '//*[@aria-expanded="false" and @role="button" and @tabindex="0" and @jsaction[starts-with(., "pane.openhours")]]'
FNB_OPENING_HOURS_BTN_RIGHT_XPATH = '//*[@data-item-id="oh"]'
FNB_OPENING_HOURS_DETAILS_XPATH = '//*[@aria-label[contains(., "Hide open hours for the week")]]'
FNB_OPENING_HOURS_BACK_BTN_XPATH = '//*[@aria-label="Back"]'
FNB_OPENING_HOURS_CLOSED_XPATH = '//span[@class=" fCEvvc"]' #Temporarily Closed / #Permanently Closed
FNB_LOCATED_IN_XPATH = '//*[@data-item-id="locatedin"]'
FNB_ADDRESS_XPATH = '//*[@data-item-id[starts-with(., "address")]]'
FNB_PHONE_XPATH = '//*[@data-item-id[starts-with(., "phone")]]'
FNB_MENU_XPATH = '//*[@data-item-id[starts-with(., "menu")]]'
FNB_WEBSITE_XPATH = '//*[@data-item-id[starts-with(., "authority")]]'
FNB_ORDER_XPATH = '//*[@aria-label[starts-with(., "Place an order")]]'

In [118]:
# Initialize Variables
fnb_list = []
fnb_list_element = []

In [119]:
def perform_search(search_input):

    # Locate the search box element
    
    search_box_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, SEARCH_BOX_INPUT_XPATH))
    )
    search_box_element.clear()
    time.sleep(0.5)
    search_box_element.send_keys(search_input + Keys.ENTER)

    # Scrolling the bar here
    # Reference: https://stackoverflow.com/questions/72543054/how-to-scroll-down-google-maps-using-selenium-python

    keepScrolling=True
    while(keepScrolling):
        side_bar_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, RESULT_LIST_XPATH))
        )
        side_bar_element.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        side_bar_element.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        end_of_list_element = driver.find_element(By.TAG_NAME, "html").get_attribute('outerHTML')
        if(end_of_list_element.find("You've reached the end of the list.") != -1):
            keepScrolling = False
            


In [120]:
def convert_opening_hours_to_dict(opening_hours):

    day_parts = opening_hours.replace('\u202f', ' ').replace('. Hide open hours for the week', '').split(';')

    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    open_hours_dict = {
        part.split(',')[0].strip(): part.split(',', 1)[1].strip()
        for part in day_parts
    }

    sorted_open_hours_dict = {day: open_hours_dict[day] for day in days_order if day in open_hours_dict}

    return sorted_open_hours_dict


In [121]:
def get_fnb_details(fnb_list_element,category,area):
    for index, fnb in enumerate(fnb_list_element): #reversed(fnb_list_element):
        try:
            name = fnb.get_attribute('aria-label')
            google_map_url = fnb.get_attribute('href')
            fnb.click()
            time.sleep(1)

        except Exception:
            print(f"Error occured while retrieving fnb details from fnb at index {index}, retrying...")
            fnb_list_element = driver.find_elements(By.XPATH, FNB_DETAILS_XPATH)
            time.sleep(1)
            fnb = fnb_list_element[index]
            name = fnb.get_attribute('aria-label')
            google_map_url = fnb.get_attribute('href')
            fnb.click()
            time.sleep(1)

        # opening_hours = ''
        located_in = ''
        phone = ''
        menu_url = ''
        website_url = ''
        order_url = ''
        address = ''
        # try:
        #     # CASE 1: OPENING HOURS BUTTON IS POINTING DOWN
        #     opening_hours_btn_elem = fnb.find_element(By.XPATH, FNB_OPENING_HOURS_BTN_DOWN_XPATH)
        #     time.sleep(1)
        #     opening_hours_btn_elem.click()
        #     time.sleep(1)
        #     opening_hours_elem = fnb.find_element(By.XPATH, FNB_OPENING_HOURS_DETAILS_XPATH)
        #     opening_hours = opening_hours_elem.get_attribute('aria-label')
        #     opening_hours_dict = convert_opening_hours_to_dict(opening_hours)

        # except NoSuchElementException:

        #     # CASE 2: OPENING HOURS BUTTON IS POINTING RIGHT
        #     try:
        #         opening_hours_btn_elem = fnb.find_element(By.XPATH, FNB_OPENING_HOURS_BTN_RIGHT_XPATH)
        #         time.sleep(1)
        #         opening_hours_btn_elem.click()
        #         time.sleep(1)
        #         opening_hours_elem = fnb.find_element(By.XPATH, FNB_OPENING_HOURS_DETAILS_XPATH)
        #         opening_hours = opening_hours_elem.get_attribute('aria-label')
        #         opening_hours_dict = convert_opening_hours_to_dict(opening_hours)
        #         driver.find_element(By.XPATH, FNB_OPENING_HOURS_BACK_BTN_XPATH).click()

        #     except NoSuchElementException:

        #         # CASE 3: FNB IS TEMPORARILY CLOSED OR PERMANENTLY CLOSED
        #         try:
        #             opening_hours_dict = fnb.find_element(By.XPATH, FNB_OPENING_HOURS_CLOSED_XPATH).text

        #         except NoSuchElementException:
                    
        #             # CASE 4: NO OPENING HOURS STATED
        #             opening_hours_dict = 'Unknown'
        #         pass
        #     pass
        try:
            address = fnb.find_element(By.XPATH, FNB_ADDRESS_XPATH).text
        except Exception: #NoSuchElementException
            address = None
            pass

        try:
            located_in = fnb.find_element(By.XPATH, FNB_LOCATED_IN_XPATH).text
        except Exception:
            located_in = None
            pass

        try:
            phone = fnb.find_element(By.XPATH, FNB_PHONE_XPATH).text
        except Exception:
            phone = None
            pass

        try:
            menu_url = fnb.find_element(By.XPATH, FNB_MENU_XPATH).get_attribute('href')
        except Exception:
            menu_url = None
            pass 

        try:
            website_url = fnb.find_element(By.XPATH, FNB_WEBSITE_XPATH).get_attribute('href')
        except Exception:
            website_url = None
            pass 
        
        try:
            order_url = fnb.find_element(By.XPATH, FNB_ORDER_XPATH).get_attribute('href')
        except Exception:
            order_url = None
            pass 

        fnb_details = {
            "Name": name,
            "Category": category,
            "Area": area,
            "Located In": located_in,
            "Address": address,
            #"Opening Hours": opening_hours_dict,
            "Phone": phone,
            "Google Map Url": google_map_url,
            "Menu Url": menu_url,
            "Website Url": website_url,
            "Order Url": order_url
        }
        print(fnb_details)
        #global fnb_list
        fnb_list.append(fnb_details)


In [122]:
def iterate_category(cateogry_list, area_list):
    for category in cateogry_list:
        for area in area_list:
            search_input = f'{category} near {area}'
            perform_search(search_input)
            time.sleep(3)
            global fnb_list_element
            fnb_list_element = driver.find_elements(By.XPATH, FNB_DETAILS_XPATH)
            #print(len(result_element))
            
            get_fnb_details(fnb_list_element,category,area)
            #break
        #break

In [123]:
iterate_category(FNB_CATEGORY_LIST, AREA_LIST)


{'Name': 'Jyu Raku', 'Category': 'Japanese Food', 'Area': 'Subang Jaya', 'Located In': None, 'Address': '13, Jalan SS 15/5a, Ss 15, 47500 Subang Jaya, Selangor', 'Phone': '03-5633 3819', 'Google Map Url': 'https://www.google.com/maps/place/Jyu+Raku/data=!4m7!3m6!1s0x31cc4c5dfeade1f1:0x4908d0c6e17208f0!8m2!3d3.079151!4d101.59268!16s%2Fg%2F1tfpg9nf!19sChIJ8eGt_l1MzDER8Ahy4cbQCEk?authuser=0&hl=en&rclk=1', 'Menu Url': None, 'Website Url': 'http://facebook.com/JYURAKUSS15/', 'Order Url': None}
{'Name': 'Shin Zushi Usj/ Subang', 'Category': 'Japanese Food', 'Area': 'Subang Jaya', 'Located In': None, 'Address': '19-G, 19-G, Jalan USJ 10/1, Taipan USJ, 47620 Subang Jaya, Selangor', 'Phone': '014-950 9954', 'Google Map Url': 'https://www.google.com/maps/place/Shin+Zushi+Usj%2F+Subang/data=!4m7!3m6!1s0x31cc4dcc86063ccf:0x938e20691477d7d6!8m2!3d3.0477583!4d101.5853412!16s%2Fg%2F11j2z965f6!19sChIJzzwGhsxNzDER1td3FGkgjpM?authuser=0&hl=en&rclk=1', 'Menu Url': None, 'Website Url': 'http://facebook.co

In [124]:
driver.quit()


In [125]:
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)
df = pd.DataFrame(fnb_list)

counts = df['Name'].value_counts()
duplicates = counts[counts > 1].reset_index().values.tolist()
if duplicates:
    for name, count in duplicates:
        print(f'"{name}", {count}')

df.to_csv('csv_datanb_list_part_1.csv',mode='a',index=False,header=False,encoding='utf-8')


"The Carpenter's Daughter Artisan Bakery & Cafe", 2
"The Coffee Bean & Tea Leaf", 2
"YODmilk.Shop", 2
"Kaliko Coffee", 2
"Dáo desserts SS15", 2
"Upstairs Café", 2
"The Soybean Factory @ SS15 Subang Jaya", 2
"The Crusty Nest", 2
"Vanilla Mille Crepe SS15", 2


In [126]:
df

,Name,Category,Area,Located In,Address,Phone,Google Map Url,Menu Url,Website Url,Order Url
0,Jyu Raku,Japanese Food,Subang Jaya,None,"13, Jalan SS 15/5a, Ss 15, 47500 Subang Jaya, ...",03-5633 3819,https://www.google.com/maps/place/Jyu+Raku/dat...,None,http://facebook.com/JYURAKUSS15/,None
1,Shin Zushi Usj/ Subang,Japanese Food,Subang Jaya,None,"19-G, 19-G, Jalan USJ 10/1, Taipan USJ, 47620 ...",014-950 9954,https://www.google.com/maps/place/Shin+Zushi+U...,None,http://facebook.com/shinzushi.usj,None
2,Kin Ryu Tei (金龍亭),Japanese Food,Subang Jaya,None,"No.3, Jalan SS 15/5a, Ss 15, 47500 Subang Jaya...",03-5612 3668,https://www.google.com/maps/place/Kin+Ryu+Tei+...,None,https://www.facebook.com/kinryutei/,https://www.foodpanda.my/restaurant/o5vs/kin-r...
3,Ryoshi Izakaya 漁師居酒屋,Japanese Food,Subang Jaya,None,"55, Jln SS 15/4E, Ss 15, 47500 Subang Jaya, Se...",010-201 0200,https://www.google.com/maps/place/Ryoshi+Izaka...,https://ryoshiizakaya-restaurant.business.site/,https://www.facebook.com/RyoshiIzakayaSubang/,None
4,Tai Syou Japanese Restaurant,Japanese Food,Subang Jaya,None,"57, Jalan SS 15/5a, Ss 15, 47500 Subang Jaya, ...",03-5612 3318,https://www.google.com/maps/place/Tai+Syou+Jap...,None,None,None
...,...,...,...,...,...,...,...,...,...,...
328,Curry Chicken Bun Subang Jaya,Dessert,Subang Jaya,None,"Pesiaran, Persiaran Subang Permai, USJ1, 47500...",014-722 1597,https://www.google.com/maps/place/Curry+Chicke...,None,http://www.twinklejelly.com/,None
329,BahWang Traditional Dessert 霸王传统糖水,Dessert,Subang Jaya,None,"No. 31, Tingkat, 1, Jalan USJ 10/1e, Taipan Bu...",None,https://www.google.com/maps/place/BahWang+Trad...,None,None,https://www.foodpanda.my/restaurant/vyk6/bahwa...
330,SKIN DESSERT DAN NURFELLA BEAUTY SHAH ALAM,Dessert,Subang Jaya,None,"Jalan Pinggiran USJ 2/1, Taman Pinggiran Usj, ...",017-905 9614,https://www.google.com/maps/place/SKIN+DESSERT...,None,https://skin-dessert-shah-alam.business.site/?...,None
331,Leong Sweet Dessert Kajang,Dessert,Subang Jaya,None,"Jalan PJS 11/9, Bandar Sunway, 47500 Subang Ja...",011-3173 0013,https://www.google.com/maps/place/Leong+Sweet+...,None,None,None
